In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

# Function to compute RMSSD
def compute_rmssd(intervals):
    if len(intervals) < 2:
        return 0
    nn_intervals = np.diff(intervals)
    return np.sqrt(np.mean(nn_intervals**2))

# Function to compute SDNN
def compute_sdnn(intervals):
    if len(intervals) < 2:
        return 0
    return np.std(np.abs(np.diff(intervals)))

# Function to process the data
def process_data(input_file_path, output_file_path, video_emotion, ibi_column='IBI PPG ALG'):
    # Read the input data
    df = pd.read_csv(input_file_path)

    # Group peaks by emotion
    emotion_peaks = {emotion: [] for emotion in set(video_emotion.values())}
    for stimulus, emotion in video_emotion.items():
        # Filter rows where SourceStimuliName is stimulus
        filtered_df = df[df['SourceStimuliName'] == stimulus]

        # Convert IBI column to numeric and drop NaN
        ibis = pd.to_numeric(filtered_df[ibi_column], errors='coerce').dropna()

        # Find peaks
        peaks, _ = find_peaks(ibis)
        positive_peaks = ibis.iloc[peaks].values
        positive_peaks = positive_peaks[positive_peaks > 0]

        emotion_peaks[emotion].extend(positive_peaks)

    # Compute RMSSD and SDNN for each emotion
    emotion_rmssd = {emotion: compute_rmssd(peaks) for emotion, peaks in emotion_peaks.items()}
    emotion_sdnn = {emotion: compute_sdnn(peaks) for emotion, peaks in emotion_peaks.items()}

    # Create a DataFrame for results
    data = [[emotion, emotion_rmssd[emotion], emotion_sdnn[emotion]] for emotion in video_emotion.values()]
    results_df = pd.DataFrame(data, columns=["Emotion", "RMSSD", "SDNN"])
    results_df.fillna(0, inplace=True)

    # Save the results to Google Drive
    results_df.to_csv(output_file_path, index=False)

# Set your file paths and video_emotion dictionary
input_file_path = '/content/drive/My Drive/OutputVF/interpolated_vF_16.csv'  # Path to your input file
output_file_path = '/content/drive/My Drive/RmssdVf/RmSn_vF_16.csv'  # Path to save your output file

video_emotion = {
    "A1": "A1_LP",
    "A2": "A2_LP",
    "A3": "A3_LP",
    "A4": "A4_LP",
    "A": "A_HN",
    "B": "B_HN",
    "C": "C_LN",
    "F": "F_HN",
    "G": "G_HP",
    "H": "H_HP",
    "J": "J_Ne",
    "K": "K_Ne",
    "M": "M_LN",
    "N": "N_LN",
    "O": "O_LN",
    "P": "P_HP",
    "Q": "Q_HP",
    "U": "U_Ne",
    "V": "V_Ne",
    "W": "W_HN"
}

# Process the data
process_data(input_file_path, output_file_path, video_emotion)

